In [1]:
import json
import io
from io import BytesIO
import os
import pandas as pd
import csv
from csv import reader
import requests
import pycurl
import certifi
import time
from datetime import datetime, date, timedelta
from acrcloud.recognizer import ACRCloudRecognizer
from acrcloud.recognizer import ACRCloudRecognizeType
from bs4 import BeautifulSoup
import shutil
from shutil import copyfile
import pydub
from pydub.playback import play
from pydub import AudioSegment
import ffmpeg
from functools import partial
from itertools import repeat
from itertools import product
import multiprocessing
from multiprocessing import Pool, freeze_support
import glob

In [2]:
workindir = r'/Users/michaelfelzan/Desktop/GEO FM'


acrcloud_accesskey = '4554490a1c83c19ea6745ed6bfbbea7d'
acrcloud_secretkey = 'HcoVKztgUltoNAwlPxpWoIJzt86HoKk9KTiByPTl'

acrcloud_config = {
    'host':'identify-eu-west-1.acrcloud.com',
    'access_key': acrcloud_accesskey,
    'access_secret': acrcloud_secretkey,
    'recognize_type': ACRCloudRecognizeType.ACR_OPT_REC_AUDIO, # could be 'humming audio' as well
    'debug':False,
    'timeout':5 # seconds
}

ACR_recognizer = ACRCloudRecognizer(acrcloud_config)

In [3]:
client_key = 'gQylLmyvGPfyGEvmFyRx'
client_secret = 'KJnItkPttweOtFSkqwAiTTCWJsOnIobH'

In [4]:
os.chdir(r'/Users/michaelfelzan/Desktop/Geography-of-Radio/Data')

In [5]:
os.getcwd()

'/Users/michaelfelzan/Desktop/Geography-of-Radio/Data'

In [6]:
os.listdir()

['MASTER_CSV.csv',
 'PLACES_DescendingCountriesCSV.csv',
 '.DS_Store',
 'WashingtonStations_filled.csv',
 'sample_mp3_1.mp3',
 'sample_mp3_2.mp3',
 'WashingtonStations_OnlyFM.csv',
 'us_4.csv',
 'WASH STATIONS',
 'US_STATIONS_3.csv',
 'fatal_error_wash.txt',
 'US_STATIONS.csv',
 'WashStationInfoTables']

In [7]:
us3_df = pd.read_csv('US_STATIONS_3.csv')
us3_df.head()  

,Country,CityState,City,StateAbv,StationName,RG_ID,RG_LC,CS,RG_URL,URL,CityCords
0,United States,Los Angeles CA,NaN,NaN,101 Smooth Jazz,zFDSxGwY,/listen/-101-smooth-jazz/zFDSxGwY,xxxx,https://radio.garden/api/ara/content/listen/zF...,https://streaming.live365.com/b22139_128mp3,"[-118.24368, 34.052235]"
1,United States,Los Angeles CA,NaN,NaN,113FM,4KJ_uzy0,/listen/radio-113fm/4KJ_uzy0,xxxx,https://radio.garden/api/ara/content/listen/4K...,http://113fm-edge1.cdnstream.com/1730_128,"[-118.24368, 34.052235]"
2,United States,Los Angeles CA,NaN,NaN,88.5 FM KCSN,SsUyqJaN,/listen/kcsn/SsUyqJaN,KCSN,https://radio.garden/api/ara/content/listen/Ss...,http://130.166.82.184:8000/;,"[-118.24368, 34.052235]"
3,United States,Los Angeles CA,NaN,NaN,9128.live,hacyg6SN,/listen/radio-9128-live/hacyg6SN,xxxx,https://radio.garden/api/ara/content/listen/ha...,https://streams.radio.co/s0aa1e6f4a/listen,"[-118.24368, 34.052235]"
4,United States,Los Angeles CA,NaN,NaN,98.5 Flex FM,t4dqOINA,/listen/radio-98-5-flex-fm/t4dqOINA,xxxx,https://radio.garden/api/ara/content/listen/t4...,https://streaming.live365.com/a23768,"[-118.24368, 34.052235]"


## NEXT CELL IS A ONE-TIME RUN TO CREATE A CSV :

In [ ]:
with open('us_4.csv','w',newline='') as csvfile:
    fieldnames = ['Country',
                  'CityState',
                  'City',
                  'StateAbv',
                  'StationName',
                  'RG_ID',
                  'RG_LC',
                  'CS',
                  'RG_URL',
                  'URL',
                  'CityCords']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    with open('US_STATIONS_3.csv', 'r') as read_obj:
        csv_reader = reader(read_obj)
        header = next(csv_reader)
        if header != None:
            for row in csv_reader:
                iter_citystate = row[1]
                iter_split = iter_citystate.split()
                stateabv = iter_split[-1]
                citywspace = iter_citystate.split(stateabv)[0]
                isod_city = citywspace[:-1]
                
                writer.writerow({
                    'Country' : row[0],
                    'CityState' : row[1],
                    'City' : isod_city,
                    'StateAbv' : stateabv,
                    'StationName' : row[4],
                    'RG_ID' : row[5],
                    'RG_LC' : row[6],
                    'CS' : row[7],
                    'RG_URL' : row[8],
                    'URL' : row[9],
                    'CityCords' : row[10]
                })

In [8]:
WA_FM_CSV = r'/Users/michaelfelzan/Desktop/Geography-of-Radio/Data/WashingtonStations_OnlyFM.csv'

source_CSV = WA_FM_CSV

In [9]:
failed_wash_infoget = r'/Users/michaelfelzan/Desktop/GEO FM/FailedWashStationsInfoGetter.txt'

In [10]:
def RadioTowerInfoGetter(callsign):
    """Function that requests the HTML of
    the radio-locator.com webpage for whatever 
    radio station callsign is inputted into the function.
    
    The function parses the HTML to return a dictionary
    containing the radio stations ERP, lat/long,
    HAAT, HAGL, and HASL.
    
    Parameters
    ------------
    callsign : str
        Name of station callsign (eg. KEXP)
    """
    print(f"Attempting to gather info from:  {callsign}...")
    
    radiolocheaders = {
        'sec-ch-ua' : '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
        'sec-ch-ua-mobile' : '?0',
        'Upgrade-Insecure-Requests' : '1',
        'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'
    }
    locatorURL_1 = f'https://radio-locator.com/cgi-bin/url?call={callsign}&service=FM'
    locatorURL_2 = f'https://radio-locator.com/cgi-bin/finder?call={callsign}&x=0&y=0&sr=Y&s=C'
    
    radiolocreq = requests.get(locatorURL_1,
                               headers=radiolocheaders)
    stationHTML = radiolocreq.content.decode('utf-8')
    stationsoup = BeautifulSoup(stationHTML)
    techvalues = stationsoup.find_all("td",
                                      class_='tech_value')
    
    if techvalues == []:
        print(f"No radio tower info could be retrieved for station {callsign}")       
        print("Trying other request URL...waiting 20 seconds before retry...")
        time.sleep(20)
        radiolocreq = requests.get(locatorURL_2,
                                   headers=radiolocheaders)
        stationHTML = radiolocreq.content.decode('utf-8')
        stationsoup = BeautifulSoup(stationHTML)
        techvalues = stationsoup.find_all("td",
                                          class_='tech_value')
        if techvalues == []:
            print("Second URL method failed. Stationed logged in 'fails' .txt")
            with open(failed_wash_infoget, "a+") as file_object:
                file_object.seek(0)
                data = file_object.read(100)
                if len(data) > 0 :
                    file_object.write("\n")
                file_object.write(f"{callsign}")
                
            return False
        
        else:
            ERP = []
            coords = []
            heights = []
            # Parsing the HTML:
            for item in techvalues:
                for characters in item:
                    for sub in characters:
                        if '" N' in sub:
                            coords.append(sub)
                if 'Watts' in characters:
                    ERP.append(characters)
                elif 'meters' in characters:
                    heights.append(characters)
                else:
                    pass
    
            try:
                itercoords = coords[0]
            except:
                print("This station didn't have coords data")
                itercoords = 'no_data'
            try:
                iterERP = ERP[0]
            except:
                print("This station didn't have ERP data")
                iterERP = 'no_data'
            try:
                iterHAAT = heights[0]
            except:
                print("This station didn't have HAAT data")
                iterHAAT = 'no_data'
            try:
                iterHAGL = heights[1]
            except:
                print("This station didn't have HAGL data")
                iterHAGL = 'no_data'
            try:
                iterHASL = heights[2]
            except:
                print("This station didn't have HASL data")
                iterHASL = 'no_data'
    
            towerinfo = {
                'ERP' : iterERP,
                'Coords' : itercoords,
                'HAAT' : iterHAAT,
                'Height Above Ground Level' : iterHAGL,
                'Height Above Sea Level' : iterHASL
            }
    
            return towerinfo
    
    else:
        ERP = []
        coords = []
        heights = []
        # Parsing the HTML:
        for item in techvalues:
            for characters in item:
                for sub in characters:
                    if '" N' in sub:
                        coords.append(sub)
            if 'Watts' in characters:
                ERP.append(characters)
            elif 'meters' in characters:
                heights.append(characters)
            else:
                pass
    
        try:
            itercoords = coords[0]
        except:
            print("This station didn't have coords data")
            itercoords = 'no_data'
        try:
            iterERP = ERP[0]
        except:
            print("This station didn't have ERP data")
            iterERP = 'no_data'
        try:
            iterHAAT = heights[0]
        except:
            print("This station didn't have HAAT data")
            iterHAAT = 'no_data'
        try:
            iterHAGL = heights[1]
        except:
            print("This station didn't have HAGL data")
            iterHAGL = 'no_data'
        try:
            iterHASL = heights[2]
        except:
            print("This station didn't have HASL data")
            iterHASL = 'no_data'
    
        towerinfo = {
            'ERP' : iterERP,
            'Coords' : itercoords,
            'HAAT' : iterHAAT,
            'Height Above Ground Level' : iterHAGL,
            'Height Above Sea Level' : iterHASL
        }
    
        towerinfo = {
            'ERP' : iterERP,
            'Coords' : itercoords,
            'HAAT' : iterHAAT,
            'Height Above Ground Level' : iterHAGL,
            'Height Above Sea Level' : iterHASL
        }
    
        return towerinfo

In [11]:
def StationInfoTxtWriter(stationfolderpath, call_sign):
    """Function that writes the dictionary return
    from RadioTowerInfoGetter() for a station 
    to a .txt file in that station's corresponding
    folder (only if the RadioTowerInfoGetter()
    successfully returns tower info).
    
    Parameters
    ------------
    stationfoldername : str
        Name of a radio stations folder (eg. CA_KALX)
    """
    towerinfodict = RadioTowerInfoGetter(call_sign)
    if towerinfodict == False:
        #print("yes, it failed")
        pass
    else:
        infofilename = os.path.join(stationfolderpath,
                                    f"{call_sign}_towerinfo.txt")
        textyfile = open(infofilename,
                    "w+")
        textyfile.write("{\n")
        for k in towerinfodict.keys():
            textyfile.write("'{}':'{}'\n".format(k, towerinfodict[k]))
        textyfile.write("}")
        textyfile.close()
        print(f"Sucessfully wrote {call_sign}_towerinfo.txt")

In [12]:
def dms_to_dd(d, m, s):
    """Function that converts degrees minutes seconds
    into decimal degrees.
    
    Parameters
    ------------
    d : str (tho doesnt matter--converted to float in function)
        degree
    m : str (tho doesnt matter--converted to float in function)
        minutes
    s : str (tho doesnt matter--converted to float in function)
        seconds
    """
    dd = d + float(m)/60 + float(s)/3600
    return round(dd, 6)

In [13]:
def DMS_Isolator(dms_string):
    """function that isolates the degree, minute, and second 
    components  of the DMS return from the StationInfoTxtWriter(),
    so that they may be separately inputted into the
    dms_to_dd() function parameters
    """
    degree = dms_string.split("°")[0]
    minutefirstsplit = dms_string.split("'")[0]
    minute = minutefirstsplit.split("° ")[1]
    secondfirstsplit = dms_string.split('"')[0]
    second = secondfirstsplit.split("' ")[1]
    
    return [degree,minute,second]

In [14]:
class Station:
    def __init__(self, sourceCSV, callsign):
        self.sourceCSV = sourceCSV
        self.callsign = callsign
        
        with open(sourceCSV, 'r') as read_obj:
            csv_reader = reader(read_obj)
            header = next(csv_reader)
            if header != None:
                for row in csv_reader:
                    if row[7] == callsign:
                        st_country = row[0]
                        st_city = row[2]
                        st_state = row[3]
                        st_url = row[10]
                        st_citycords = row[11]
                        
        self.country = st_country
        self.city = st_city
        self.state = st_state
        self.url = st_url
        self.citycords = st_citycords
    
    def Sampler(self, url, outpathname):
        audio_input = ffmpeg.input(url)
        audio_output = ffmpeg.output(audio_input,
                                     outpathname,
                                     **{'b:a': '128k'},
                                     ss=35,
                                     t=10)
        #print(audio_output)
        try:
            audio_output.run()
        except:
            print("Error - Sampler could not work on URL stream. Skipping station.")
            return False
    
    def RouteToACRCloud(self, mp3):
        """Function that routes an mp3 file (path) to the
        ACRCloud API, in order to return the song name info.
        
        Parameters
        ------------
        mp3 : str
            Name of mp3 file (path)
            """
        buf = open(mp3,'rb').read()
        # start second will be 0; will record sampled mp3 for 10 seconds
        songread_output = ACR_recognizer.recognize_by_filebuffer(buf, 0)
        return songread_output
    
    
    def RollRadio(self, url, callsign):
        timerightnow = datetime.now()
        formattedtime = timerightnow.strftime("%Y_%m_%d_T%H_%M_%S")
        
        input_stream = url
        input_mp3name = f'{callsign}'+f'{formattedtime}'+'.mp3'
        itersongpath = os.path.join(r'/Users/michaelfelzan/Desktop/GEO FM',
                                    input_mp3name)
        
        print(f'~~Now recording... {input_stream}')
        samplerreturn = self.Sampler(url, itersongpath)
        err = True
        if samplerreturn == False:
            pass
        else:
            acrcloud_output = self.RouteToACRCloud(itersongpath)
            songread_dict = json.loads(acrcloud_output)
    
            if songread_dict['status']['msg'] == 'No result':
                print(f'No ACR return from {input_mp3name}')
                pass
            elif songread_dict['status']['msg'] == 'May Be Mute':
                print(f'No ACR return from {input_mp3name}')
                pass
            elif songread_dict['status']['msg'] == 'Decode Audio Error':
                print(f'No ACR return from {input_mp3name}')
                pass
            elif songread_dict['status']['msg'] == 'requests limit exceeded, please upgrade your account':
                print('ACRCLOUD REQUESTS LIMITS EXCEEDED, STOP PROGRAM')
                pass
            else:
                try:
                    songtitle = songread_dict['metadata']['music'][0]['title']
                except:
                    print("Error in 'Title' ACR return metadata...skipping station")
                    err = False
                if err == False:
                    pass
                else:
                    try:
                        songartist = songread_dict['metadata']['music'][0]['artists'][0]['name']
                    except:
                        print("Error in 'Artist' ACR return metadata...skipping station")
                        err = False
                    if err == False:
                        pass
                    else:
                        try:
                            songalbum = songread_dict['metadata']['music'][0]['album']['name']
                        except:
                            print("Error in 'Album' ACR return metadata...skipping station")
                            err = False
                            pass
                if err == False:
                    pass
                else:
                    ACR_return = f'{input_mp3name} ~ song : {songtitle} by {songartist} from album {songalbum}'
    
                    percent20song = songtitle+'%20'+songartist+'%20'+songalbum
                    discogs_input = percent20song.replace(" ", "%20")
                    discogs_req = ('https://api.discogs.com/database/search?q=' + 
                               discogs_input + 
                               '&key=' + 
                               client_key + 
                               '&secret=' +
                               client_secret)
                    discogs_req_obj = requests.get(discogs_req)
                    discogs_songinfo_dict = json.loads(discogs_req_obj.content.decode('utf-8'))
        
                    if discogs_songinfo_dict['results'] == []:
                        print(f'No Discogs return from {input_mp3name}')
                        pass
                    else:
                        try:
                            style_list = discogs_songinfo_dict['results'][0]['style']
                        except:
                            style_list = 'no_data'
                        try:
                            genre_list = discogs_songinfo_dict['results'][0]['genre']
                        except:
                            genre_list = 'no_data'
                        
                        if style_list == 'no_data':
                            syleliststring = 'no_data'
                        else:
                            styleliststring = ','.join(style_list)
                        if genre_list == 'no_data':
                            genreliststring  = 'no_data'
                        else:
                            genreliststring = ','.join(genre_list)
            
                        Discogs_return = f'Discogs returned the genre(s): {styleliststring}'
                
                        parent_dir = r'/Users/michaelfelzan/Desktop/GEO FM/WASH STATIONS'
                
                        itertextpath = os.path.join(parent_dir,
                                                    callsign,
                                                    f'{callsign}.txt')
            
                        with open(itertextpath, "a+") as file_object:
                            file_object.seek(0)
                            data = file_object.read(100)
                            if len(data) > 0 :
                                file_object.write("\n")
                            file_object.write((input_mp3name+
                                               ','+
                                               'song='+songtitle+','+
                                               'artist='+songartist+','+
                                               'album='+songalbum+','+
                                               ####'genres='+genreliststring+','+
                                               'styles='+styleliststring))
                
                        print(f"Returned:{[ACR_return,Discogs_return]}")
                
                os.remove(itersongpath)


In [ ]:
kexptower = RadioTowerInfoGetter('KMRE')
kexptower

In [ ]:
kkxatower = RadioTowerInfoGetter('KKXA')
kkxatower

In [15]:
fails_txt = r'/Users/michaelfelzan/Desktop/GEO FM/FailedWashStationsInfoGetter.txt'
fails_list = []

with open(fails_txt, "r") as a_file:
    for line in a_file:
        stripped_line = line.strip()
        fails_list.append(stripped_line)

In [16]:
fails_list

['KOWA', 'KORE', 'KVSH']

#### One-time use code for deleting all text files !! :

In [ ]:
#parent_dir = r'/Users/michaelfelzan/Desktop/GEO FM/WASH STATIONS'

#for washcallsign in list_of_wash_callsigns:
    #iter_station = Station(WA_FM_CSV, washcallsign)
    #itertextpath = os.path.join(parent_dir,
                               #washcallsign,
                               #f'{washcallsign}.txt')
    #os.remove(itertextpath)

##### Creating 'list_of_wash_callsigns;' appending all names of Washington callsigns to list:

In [17]:
list_of_wash_callsigns = []

with open(source_CSV, 'r') as read_obj:
    csv_reader = reader(read_obj)
    header = next(csv_reader)
    if header != None:
        for row in csv_reader:
            iter_callsign = row[7]
            list_of_wash_callsigns.append(iter_callsign)

##### Making folder directories for all Washington callsigns:

In [ ]:
for washcallsign in list_of_wash_callsigns:
    iter_station = Station(WA_FM_CSV, washcallsign)
    # Directory 
    directory_name = iter_station.callsign
    
    # Parent Directory path
    parent_dir = r'/Users/michaelfelzan/Desktop/GEO FM/WASH STATIONS'
    
    # Path 
    foldpath = os.path.join(parent_dir, directory_name)

    # Create the directory 
    os.mkdir(foldpath)

##### Creating .txt files for all Washington callsigns in correct format:

In [ ]:
for washcallsign in list_of_wash_callsigns:
    itertextpath = os.path.join(parent_dir,
                               washcallsign,
                               f'{washcallsign}.txt')
    #print(itertextpath)
    
    with open(itertextpath, "w") as f:
        f.write(f'{washcallsign}')
    
    
for washcallsign in list_of_wash_callsigns:
    iter_station = Station(WA_FM_CSV,
                           washcallsign)
    itertextpath = os.path.join(parent_dir,
                               washcallsign,
                               f'{washcallsign}.txt')
    with open(itertextpath, "a+") as file_object:
        file_object.seek(0)
        # If file is not empty then append '\n'
        data = file_object.read(100)
        if len(data) > 0 :
            file_object.write("\n")
        # Append text at the end of file
        file_object.write(f"stream={iter_station.url}")
        file_object.write("\n")

In [ ]:
#list_of_wash_callsigns

In [ ]:
KEGX = Station(WA_FM_CSV, 'KEGX')

In [ ]:
KEGX.RollRadio(KEGX.url, KEGX.callsign)

In [ ]:
parent_dir = r'/Users/michaelfelzan/Desktop/GEO FM/WASH STATIONS'

for cs in fails_list:
    
    iterfoldpathy = os.path.join(parent_dir,
                                cs)
    # StationInfoTxtWriter(stationfolderpath, call_sign):
    StationInfoTxtWriter(iterfoldpathy,
                         cs)
    time.sleep(20)

In [18]:
parent_dir = r'/Users/michaelfelzan/Desktop/GEO FM/WASH STATIONS'
os.chdir(parent_dir)
wash_callsign_folds = []
for thing in os.listdir():
    if thing == '.DS_Store':
        pass
    else:
        wash_callsign_folds.append(thing)

In [19]:
wash_callsign_folds

['KPBX',
 'KHUH',
 'KACS',
 'KYRS',
 'KXDD',
 'KZAX',
 'KMRE',
 'KFAE',
 'KUKN',
 'KOSW',
 'KTQA',
 'KBCS',
 'KZTM',
 'KEZE',
 'KPLW',
 'KROH',
 'KGHI',
 'KUPS',
 'KIEV',
 'KSQM',
 'KAOS',
 'KMIH',
 'KPBZ',
 'KTAH',
 'KEFA',
 'KSER',
 'KNKX',
 'KUOW',
 'KIRO',
 'KGRG',
 'KEWU',
 'KDDS',
 'KWCW',
 'KNHC',
 'KEXP',
 'KZZU',
 'KZQM',
 'KUGS',
 'KPTZ',
 'KBFG',
 'KXLY',
 'KODX',
 'KGHP',
 'KING',
 'KYYO',
 'KEGX',
 'KCMS']

In [20]:
os.chdir(r'/Users/michaelfelzan/Desktop/GEO FM')
os.getcwd()

'/Users/michaelfelzan/Desktop/GEO FM'

In [21]:
stream = os.popen('node fezCC.js 4.7 211 "250" "-" 60 "0"')
output = (stream.read()).split('\n')[0]
output

'37.699469101964276'

In [ ]:
for cs in wash_callsign_folds:
    pathtostationfold = os.path.join(parent_dir,
                                    cs)
    #itercallsign = station.split("_")[1]
    #stateabv = station.split("_")[0]
    pathtotowertxt = os.path.join(pathtostationfold,
                            f'{cs}_towerinfo.txt')
    pathtotxt = os.path.join(pathtostationfold,
                             f'{cs}.txt')
    newinfocsv = os.path.join(pathtostationfold,
                             f'{cs}_INFO.csv')
    stationfolditems = os.listdir(pathtostationfold)
    fullpathitems = []
    for item in stationfolditems:
        fullpathitems.append(os.path.join(pathtostationfold,
                                         item))
    if pathtotowertxt in fullpathitems:
        with open(pathtotowertxt) as foo:
            txtz = foo.readlines()
            for item in txtz:
                
                if 'ERP' in item:
                    wattsfirstsplit = item.split(":'")[1]
                    watts = wattsfirstsplit.split(" Watts")[0]
                    kilowatts = (float(watts.replace(',','')))/1000
                    
                elif 'Coords' in item:
                    coordsfirstsplit = item.split(":'")[1]
                    lat_dms = coordsfirstsplit.split(",")[0]
                    long_dms = (coordsfirstsplit.split(", ")[1])[:-2]
                    
                    lat_dms_list = DMS_Isolator(lat_dms)
                    long_dms_list = DMS_Isolator(long_dms)
                    
                    lat_dd = dms_to_dd(float(lat_dms_list[0]),
                                       float(lat_dms_list[1]),
                                       float(lat_dms_list[2]))
                    long_dd = dms_to_dd(float(long_dms_list[0]),
                                        float(long_dms_list[1]),
                                        float(long_dms_list[2]))
                    negative_longdd = long_dd*-1
                    
                elif 'HAAT' in item:
                    HAATfirstsplit = item.split(":'")[1]
                    HAAT_clean = HAATfirstsplit.split(" meters")[0]
                    HAAT_float = float(HAAT_clean)
        
                    stream_param = f'node fezCC.js {kilowatts} {HAAT_float} "250" "-" 60 "0"'
                    stream = os.popen(stream_param)
                    BUFF60DIST = (stream.read()).split('\n')[0]
                
        with open(newinfocsv,'w',newline='') as csvfile:
            fieldnames = ['CALLSIGN',
                          'LAT',
                          'LONG',
                          'ERP',
                          'HAAT',
                          '60dBu_DIST']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerow({
                'CALLSIGN': cs,
                'LAT' : lat_dd,
                'LONG' : negative_longdd,
                'ERP' : kilowatts,
                'HAAT' : HAAT_clean,
                '60dBu_DIST' : BUFF60DIST
            })
                
    else:
        print(f'tower info .txt does not exist for {cs}')
        pass

print("\n     *~*~~~~~**~~~~~~~~~~~~~~**~*~*\n"+
      "Successfully wrote all other station tower info CSVs."+
      "\nCSVs outputted to respective station folders."
      "\n   *~*~~~~~**~~~~~~~~~~~~~~**~*~*")

In [ ]:
stationinfopath = os.path.join(r'/Users/michaelfelzan/Desktop/GEO FM',
                               "WashStationInfoTables")
os.mkdir(stationinfopath)

In [ ]:
# COPYING FILES TO StationInfoTables FOLDER

for cs in wash_callsign_folds:
    pathtostationfold = os.path.join(parent_dir,
                                    cs,)
    newinfocsv = os.path.join(pathtostationfold,
                             f'{cs}_INFO.csv')
    stationfolditems = os.listdir(pathtostationfold)
    fullpathitems = []
    for item in stationfolditems:
        fullpathitems.append(os.path.join(pathtostationfold,
                                         item))
    if newinfocsv in fullpathitems:
        copyfile(newinfocsv,
                 os.path.join(r'/Users/michaelfelzan/Desktop/GEO FM',
                              "WashStationInfoTables",
                              f'{cs}_INFO.csv'))

In [ ]:
pathtofold = r'/Users/michaelfelzan/Desktop/GEO FM/WashStationInfoTables'
INFOconcatCSV = os.path.join(pathtofold,
                             "WASHSTATIONINFOconcat.csv") 
allFiles = glob.glob(pathtofold + "/*.csv")
allFiles.sort()
with open(INFOconcatCSV, 'wb') as outfile:
        for i, fname in enumerate(allFiles):
            with open(fname, 'rb') as infile:
                if i != 0:
                    infile.readline()  # Throw away header on all but first file
            # Block copy rest of file from input to output without parsing
                shutil.copyfileobj(infile, outfile)

print("     *~*~~~~~**~~~~~~~~~~~~~~**~*~*\n"
      "All station tower info CSVs (that exist) have"+
      "\n      been successfully merged together"
      "\n  *~*~~~~~**~~~~~~~~~~~~~~**~*~*")

In [22]:
wash_stn_info_concat = os.path.join(r'/Users/michaelfelzan/Desktop/GEO FM',
                                    "WashStationInfoTables",
                                    "WASHSTATIONINFOconcat.csv")

wash_stn_info_df = pd.read_csv(wash_stn_info_concat)
wash_stn_info_df

,CALLSIGN,LAT,LONG,ERP,HAAT,60dBu_DIST
0,KACS,46.730556,-123.025833,6.000,57.00,21.947963
1,KAOS,47.015833,-122.917222,1.250,74.00,16.686530
2,KBCS,47.543889,-122.109167,1.800,389.00,40.247137
3,KBFG,47.667500,-122.355000,0.009,95.94,5.546802
4,KCMS,47.544167,-122.108333,54.000,385.00,72.279685
5,KDDS,47.312500,-123.372222,64.000,742.00,92.461470
6,KEFA,47.382778,-120.293333,0.100,-71.00,5.636485
7,KEGX,46.099167,-119.128889,100.000,424.40,81.532897
8,KEWU,47.578611,-117.298333,10.000,429.00,57.879066
9,KEXP,47.615833,-122.308889,4.700,211.00,37.699469


In [ ]:
#while True:
    #for cs in wash_callsign_folds:
        #iter_satysh = Station(WA_FM_CSV, cs)
        #iter_satysh.RollRadio(iter_satysh.url,
                             # iter_satysh.callsign)

In [23]:
KACS = Station(WA_FM_CSV, 'KACS')
KZZU = Station(WA_FM_CSV, 'KZZU')
KEXP = Station(WA_FM_CSV, 'KEXP')
KYRS = Station(WA_FM_CSV, 'KYRS')

In [39]:
def multi_run_wrapper(args):
    return RadioRoller(*args)
def RadioRoller(url, callsign):
    return (globals()[f'{callsign}']).RollRadio(globals()[f'{callsign}'].url,
                                               globals()[f'{callsign}'].callsign)
if __name__ == "__main__":
    pool = Pool(3)
    startcycletime = datetime.time(datetime.now())
    timecounter = 0
    while timecounter < 899:
        pool.map(multi_run_wrapper,[(KEXP.url,KEXP.callsign),
                                    (KZZU.url,KZZU.callsign),
                                    (KYRS.url,KYRS.callsign),
                                    (KACS.url,KACS.callsign)])
        endcycletime = datetime.time(datetime.now())
        runduration = datetime.combine(date.today(), endcycletime) - datetime.combine(date.today(), startcycletime)
        rundur_sec = runduration.seconds
        if rundur_sec < 300:
            time.sleep(300 - rundur_sec)
            timecounter += 300

~~Now recording... https://kexp-mp3-128.streamguys1.com/kexp128.mp3
~~Now recording... https://15373.live.streamtheworld.com/KZZUFMAAC_SC
~~Now recording... https://www.ophanim.net:8444/s/7170
Error - Sampler could not work on URL stream. Skipping station.
~~Now recording... https://ic2.sslstream.com/kacs-fm
No ACR return from KACS2021_10_27_T12_38_10.mp3
Returned:["KEXP2021_10_27_T12_38_08.mp3 ~ song : Less Yes's, More No's by Busdriver from album RoadKill Overcoat", 'Discogs returned the genre(s): Abstract']
No ACR return from KZZU2021_10_27_T12_38_08.mp3
